In [1]:
import pandas as pd

In [2]:
med_data = pd.read_csv('../../data/claims_med_interns_2016-2018_20190304.csv')

D:\Users\vhsu\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
med = med_data[0:300]

In [4]:
# Dict: county -> urban/rural
urban_rural = {'ADAMS': 'rural', 'ASOTIN': 'rural', 'BENTON': 'urban', 'CHELAN': 'rural', 'CLALLAM': 'rural', 'CLARK': 'urban', 'COLUMBIA': 'rural', 'COWLITZ': 'rural', 'DOUGLAS': 'rural', 'FERRY': 'rural', 'FRANKLIN': 'rural', 'GARFIELD': 'rural', 'GRANT': 'rural', 'GRAYS HARBOR': 'rural', 'ISLAND': 'rural', 'JEFFERSON': 'rural', 'KING': 'urban', 'KITSAP': 'urban', 'KITTITAS': 'rural', 'KLICKITAT': 'rural', 'LEWIS': 'rural', 'LINCOLN': 'rural', 'MASON': 'rural', 'OKANOGAN': 'rural', 'PACIFIC': 'rural', 'PEND OREILLE': 'rural', 'PIERCE': 'urban', 'SAN JUAN': 'rural', 'SKAGIT': 'rural', 'SKAMANIA': 'rural', 'SNOHOMISH': 'urban', 'SPOKANE': 'urban', 'STEVENS': 'rural', 'THURSTON': 'urban', 'WAHKIAKUM': 'rural', 'WALLA WALLA': 'rural', 'WHATCOM': 'rural', 'WHITMAN': 'rural', 'YAKIMA': 'rural'}

In [5]:
def reshape(group):
    df = pd.DataFrame()

    # Demographic info
    df['member_id'] = group.head(1)['Member ID Encrypted']
    df['year'] = group.head(1)['Incurred Year']
    df['gender'] = group.head(1)['Member Gender']
    df['age'] = group.head(1)['Age']
    df['postal_code'] = group.head(1)['Zip (5-digit)']
    county = group.head(1)['County'].iloc[0]
    df['county'] = county
    if county in urban_rural:
        df['urban_rural'] = urban_rural[county]
    
    # Get all ICD codes and Rollups for this group
    icd_codes = group['Primary ICD Diagnosis Code'] + group['2nd ICD Diagnosis Code'] + group['3rd ICD Diagnosis Code'] + group['4th ICD Diagnosis Code'] + group['5th ICD Diagnosis Code'] + group['6th ICD Diagnosis Code'] + group['7th ICD Diagnosis Code'] + group['8th ICD Diagnosis Code'] + group['9th ICD Diagnosis Code'] + group['10th ICD Diagnosis Code']
    icd_rollups = group['Primary ICD Rollup'] + group['2nd ICD Rollup'] + group['3rd ICD Rollup'] + group['4th ICD Rollup'] + group['5th ICD Rollup'] + group['6th ICD Rollup'] + group['7th ICD Rollup'] + group['8th ICD Rollup'] + group['9th ICD Rollup'] + group['10th ICD Rollup']
    
    # Set binary for conditions: 1 if condition exists, 0 otherwise
    df['binary_asthma'] = 1 if any('Asthma' in x for x in icd_rollups) else 0
    df['binary_bp'] = 1 if any('I10' in x for x in icd_codes) else 0
    df['binary_cancer'] = 1 if any('Cancer' in x for x in icd_rollups) else 0
    df['binary_cardiovascular'] = 1 if any('heart' in x for x in icd_rollups) else 0
    df['binary_copd'] = 1 if any('Chronic obstructive pulmonary disease' in x for x in icd_rollups) else 0
    df['binary_diabetes'] = 1 if any('Diabetes' in x for x in icd_rollups) else 0
    df['binary_kidney'] = 1 if any('N18' in x for x in icd_codes) else 0
    df['binary_obesity'] = 1 if any('E66' in x for x in icd_codes) else 0
    df['binary_musculoskeletal'] = 1 if any('musculoskeletal' in s for s in icd_rollups) else 0
    df['binary_cholesterol'] = 1 if any('E78' in x for x in icd_codes) else 0
    #doesn't contain Bipolar disorders, Eating disorders, Psychogenic disorders, Other miscellaneous mental conditions, 
                                    #Codes related to mental health disorders, Dissociative diorders, Somatoform disorders
    mental_health = ['Anxiety disorders', 'Adjustment disorders', 'Mood disorders', 
                     'Schizophrenia and other psychotic disorders', 
                     'Suicide and intentional self-inflicted injury', 
                     'Impulse control disorders not elsewhere classified', 
                     'Screening and history of mental health and substance abuse codes', 
                    'Miscellaneous mental disorders']
    df['binary_mental_disorder'] = 1 if any(x in mental_health for x in icd_rollups) or any('G4700' in x for x in icd_codes) else 0
    
    # Initialize count columns
    df['claims_asthma'] = 0
    df['claims_bp'] = 0
    df['claims_cancer'] = 0
    df['claims_cardiovascular'] = 0
    df['claims_copd'] = 0
    df['claims_diabetes'] = 0
    df['claims_kidney'] = 0
    df['claims_obesity'] = 0
    df['claims_musculoskeletal'] = 0
    df['claims_cholesterol'] = 0
    df['claims_mental_disorder'] = 0
    df['total_claims'] = 0
    
    # Group by Claim ID to get unique claim counts
    claim_grouped = group.groupby(['Claim ID'], as_index=False)
    for claim_id, claim_group in claim_grouped:
        claim_reshape(claim_group, df)

    # Total number of conditions
    total_conditions = df['binary_kidney'] + df['binary_mental_disorder'] + df['binary_bp'] + df['binary_cardiovascular'] + df['binary_diabetes'] + df['binary_obesity'] + df['binary_cancer'] + df['binary_musculoskeletal'] + df['binary_copd'] + df['binary_cholesterol']
    df['total_conditions'] = total_conditions
    
    # Flags for nonused (claims were not filed for our conditions) and comorbidity (has more than one condition out of the ones we are interested in)
    df['flag_nonused'] = df['total_claims'].apply(lambda x: 1 if x == 0 else 0)
    df['flag_comorbidity'] = total_conditions.apply(lambda x: 1 if x > 1 else 0)

    return df

In [6]:
def claim_reshape(claim_group, df): 
    # Lists of all codes and rollups
    icd_codes = claim_group['Primary ICD Diagnosis Code'] + claim_group['2nd ICD Diagnosis Code'] + claim_group['3rd ICD Diagnosis Code'] + claim_group['4th ICD Diagnosis Code'] + claim_group['5th ICD Diagnosis Code'] + claim_group['6th ICD Diagnosis Code'] + claim_group['7th ICD Diagnosis Code'] + claim_group['8th ICD Diagnosis Code'] + claim_group['9th ICD Diagnosis Code'] + claim_group['10th ICD Diagnosis Code']
    icd_rollups = claim_group['Primary ICD Rollup'] + claim_group['2nd ICD Rollup'] + claim_group['3rd ICD Rollup'] + claim_group['4th ICD Rollup'] + claim_group['5th ICD Rollup'] + claim_group['6th ICD Rollup'] + claim_group['7th ICD Rollup'] + claim_group['8th ICD Rollup'] + claim_group['9th ICD Rollup'] + claim_group['10th ICD Rollup']
       
    if any('Asthma' in x for x in icd_rollups):
        df['claims_asthma'] += 1  
    if any('I10' in x for x in icd_codes):
        df['claims_bp'] += 1
    if any('Cancer' in x for x in icd_rollups):
        df['claims_cancer'] += 1
    if any('heart' in x for x in icd_rollups):
        df['claims_cardiovascular'] += 1
    if any('Chronic obstructive pulmonary disease' in x for x in icd_rollups):
        df['claims_copd'] += 1
    if any('Diabetes' in x for x in icd_rollups):
        df['claims_diabetes'] += 1
    if any('N18' in x for x in icd_codes):
        df['claims_kidney'] += 1
    if any('E66' in x for x in icd_codes):
        df['claims_obesity'] += 1
    if any('E78' in x for x in icd_codes):
        df['claims_cholesterol'] += 1
    if any('musculoskeletal' in x for x in icd_rollups):
        df['claims_musculoskeletal'] += 1
    mental_health = ['Anxiety disorders', 'Adjustment disorders', 'Mood disorders', 
                     'Schizophrenia and other psychotic disorders', 
                     'Suicide and intentional self-inflicted injury', 
                     'Impulse control disorders not elsewhere classified', 
                     'Screening and history of mental health and substance abuse codes', 
                    'Miscellaneous mental disorders']
    if any(x in mental_health for x in icd_rollups) or any('G4700' in x for x in icd_codes):
        df['claims_mental_disorder'] += 1

    df['total_claims'] += 1

In [7]:
# Reshape data by grouping by member and year
reshaped_med_data = med_data.groupby(['Member ID Encrypted', 'Incurred Year'], as_index=False).apply(reshape)

D:\Users\vhsu\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:924: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis, keys=keys)


In [10]:
columns = ['member_id', 'year', 'age', 'gender', 'postal_code', 'county', 'urban_rural', 'binary_asthma', 'binary_bp', 'binary_cancer', 'binary_cardiovascular', 'binary_cholesterol', 'binary_copd', 'binary_diabetes', 'binary_kidney', 'binary_mental_disorder', 'binary_musculoskeletal', 'binary_obesity', 'claims_asthma', 'claims_bp', 'claims_cancer', 'claims_cardiovascular', 'claims_cholesterol', 'claims_copd', 'claims_diabetes', 'claims_kidney', 'claims_mental_disorder', 'claims_musculoskeletal', 'claims_obesity', 'flag_comorbidity', 'flag_nonused', 'total_claims', 'total_conditions']

In [11]:
reshaped_med_data = reshaped_med_data[columns]

In [12]:
reshaped_med_data.to_csv('../../data/reshaped_med.csv', index=False)

In [13]:
reshaped_med_data.iloc[:,0:16]

member_id  year  age gender  postal_code        county  \
0     0       710087KT90EY0DZUV  2018   47      F        98335        PIERCE   
1     13      71008ADZ2VBBDM14C  2016   54      F        98032          KING   
2     14      7100UNBYH5SYEX19X  2016   63      F        98926      KITTITAS   
3     21      7100UNBYH5SYEX19X  2018   65      F        98926      KITTITAS   
4     22      71069K607Y1CQ9BV5  2016   70      M        98059          KING   
5     27      71069K607Y1CQ9BV5  2017   71      M        98059          KING   
6     88      71069K607Y1CQ9BV5  2018   72      M        98059          KING   
7     246     7107GY7V3ZWGK7FKP  2016   57      F        98520  GRAYS HARBOR   
8     310     7107GY7V3ZWGK7FKP  2017   58      F        98520  GRAYS HARBOR   
9     311     7107NGE7PLEN7UMDY  2016   26      F        98030          KING   
10    312     7107NI7V3Z1FLPJPW  2018   65      F        98037     SNOHOMISH   
11    317     7107NK607BUZW3GTE  2017   48      F        98506      THURSTON   
12    318     71092KPSENY0F15QB  2017   70      M        98855      OKANOGAN   
13    360     7109AAWZ2V1MFN916  2017   49      F        98467        PIERCE   
14    375     7109BR357BBE7WQ53  2016   43      F        98837         GRANT   
15    397     7109BR357BBE7WQ53  2017   44      F        98837         GRANT   
16    419     7109XG7SENCU8R19X  2018   55      F        98532         LEWIS   
17    427     7109ZQBCIIBL9244C  2016   41      F        98579      THURSTON   
18    428     710D3TASENP7I4ZUV  2016   45      F        99344         ADAMS   
19    432     710D4ZYBZ3JTRU581  2016   69      F        98118          KING   
20    445     710D4ZYBZ3JTRU581  2017   69      F        98118          KING   
21    502     710D4ZYBZ3JTRU581  2018   71      F        98118          KING   
22    503     710DKU1U5HH70NPJF  2016   60      F        98926      KITTITAS   
23    509     710DKU1U5HH70NPJF  2017   61      F        98926      KITTITAS   
24    513     710DKU1U5HH70NPJF  2018   62      F        98926      KITTITAS   
25    537     710DKU8KDR1CCV2N7  2016   52      F        98290     SNOHOMISH   
26    549     710N7PU1GHS2F6GTE  2016   55      F        98277        ISLAND   
27    552     710N7PU1GHS2F6GTE  2017   56      F        98277        ISLAND   
28    554     710N7PU1GHS2F6GTE  2018   58      F        98277        ISLAND   
29    561     710NXDZDY7BRQ7KQB  2017   26      F        98108          KING   
...                         ...   ...  ...    ...          ...           ...   
19233 417408  ZWUKDLQ4RBBR1VCM4  2017   58      F        98405        PIERCE   
19234 417425  ZWUKDLQ4RBBR1VCM4  2018   58      F        98405        PIERCE   
19235 417509  ZWUX147SEKW6SHJPW  2016   61      F        98801        CHELAN   
19236 417575  ZWUX67KT97FJ226E9  2016   61      F        98558        PIERCE   
19237 417577  ZWUX67KT97FJ226E9  2017   62      F        98558        PIERCE   
19238 417600  ZWV0ENE7I53U45CZH  2017   56      F        99360   WALLA WALLA   
19239 417612  ZWV0ENE7I53U45CZH  2018   57      F        99360   WALLA WALLA   
19240 417615  ZWV0U387PM4IYGTG0  2017   43      F        99362   WALLA WALLA   
19241 417625  ZWV12MV0KNCXZEX81  2016   68      F        98938        YAKIMA   
19242 417629  ZWV12MV0KNCXZEX81  2017   70      F        98938        YAKIMA   
19243 417633  ZWV12MV0KNCXZEX81  2018   71      F        98938        YAKIMA   
19244 417647  ZWV16CDAQBZQ6UVBZ  2016   66      F        98901        YAKIMA   
19245 417649  ZWV16CDAQBZQ6UVBZ  2017   67      F        98901        YAKIMA   
19246 417651  ZWV246AALG0ZYGN0N  2018   65      M        98133          KING   
19247 417668  ZWV24V3T0AURI5ZCR  2017   35      F        99362   WALLA WALLA   
19248 417673  ZWV24V3T0AURI5ZCR  2018   37      F        99362   WALLA WALLA   
19249 417674  ZWV2AHIFLGTGTZYWM  2016   68      F        98837         GRANT   
19250 417681  ZWV2AHIFLGTGTZYWM  2017   69      F        98837         GRANT   
19251 417693  ZWV2F2FL

In [14]:
pd.read_csv('../../data/reshaped_med.csv')

member_id  year  age gender  postal_code        county  \
0      710087KT90EY0DZUV  2018   47      F        98335        PIERCE   
1      71008ADZ2VBBDM14C  2016   54      F        98032          KING   
2      7100UNBYH5SYEX19X  2016   63      F        98926      KITTITAS   
3      7100UNBYH5SYEX19X  2018   65      F        98926      KITTITAS   
4      71069K607Y1CQ9BV5  2016   70      M        98059          KING   
5      71069K607Y1CQ9BV5  2017   71      M        98059          KING   
6      71069K607Y1CQ9BV5  2018   72      M        98059          KING   
7      7107GY7V3ZWGK7FKP  2016   57      F        98520  GRAYS HARBOR   
8      7107GY7V3ZWGK7FKP  2017   58      F        98520  GRAYS HARBOR   
9      7107NGE7PLEN7UMDY  2016   26      F        98030          KING   
10     7107NI7V3Z1FLPJPW  2018   65      F        98037     SNOHOMISH   
11     7107NK607BUZW3GTE  2017   48      F        98506      THURSTON   
12     71092KPSENY0F15QB  2017   70      M        98855      OKANOGAN   
13     7109AAWZ2V1MFN916  2017   49      F        98467        PIERCE   
14     7109BR357BBE7WQ53  2016   43      F        98837         GRANT   
15     7109BR357BBE7WQ53  2017   44      F        98837         GRANT   
16     7109XG7SENCU8R19X  2018   55      F        98532         LEWIS   
17     7109ZQBCIIBL9244C  2016   41      F        98579      THURSTON   
18     710D3TASENP7I4ZUV  2016   45      F        99344         ADAMS   
19     710D4ZYBZ3JTRU581  2016   69      F        98118          KING   
20     710D4ZYBZ3JTRU581  2017   69      F        98118          KING   
21     710D4ZYBZ3JTRU581  2018   71      F        98118          KING   
22     710DKU1U5HH70NPJF  2016   60      F        98926      KITTITAS   
23     710DKU1U5HH70NPJF  2017   61      F        98926      KITTITAS   
24     710DKU1U5HH70NPJF  2018   62      F        98926      KITTITAS   
25     710DKU8KDR1CCV2N7  2016   52      F        98290     SNOHOMISH   
26     710N7PU1GHS2F6GTE  2016   55      F        98277        ISLAND   
27     710N7PU1GHS2F6GTE  2017   56      F        98277        ISLAND   
28     710N7PU1GHS2F6GTE  2018   58      F        98277        ISLAND   
29     710NXDZDY7BRQ7KQB  2017   26      F        98108          KING   
...                  ...   ...  ...    ...          ...           ...   
19233  ZWUKDLQ4RBBR1VCM4  2017   58      F        98405        PIERCE   
19234  ZWUKDLQ4RBBR1VCM4  2018   58      F        98405        PIERCE   
19235  ZWUX147SEKW6SHJPW  2016   61      F        98801        CHELAN   
19236  ZWUX67KT97FJ226E9  2016   61      F        98558        PIERCE   
19237  ZWUX67KT97FJ226E9  2017   62      F        98558        PIERCE   
19238  ZWV0ENE7I53U45CZH  2017   56      F        99360   WALLA WALLA   
19239  ZWV0ENE7I53U45CZH  2018   57      F        99360   WALLA WALLA   
19240  ZWV0U387PM4IYGTG0  2017   43      F        99362   WALLA WALLA   
19241  ZWV12MV0KNCXZEX81  2016   68      F        98938        YAKIMA   
19242  ZWV12MV0KNCXZEX81  2017   70      F        98938        YAKIMA   
19243  ZWV12MV0KNCXZEX81  2018   71      F        98938        YAKIMA   
19244  ZWV16CDAQBZQ6UVBZ  2016   66      F        98901        YAKIMA   
19245  ZWV16CDAQBZQ6UVBZ  2017   67      F        98901        YAKIMA   
19246  ZWV246AALG0ZYGN0N  2018   65      M        98133          KING   
19247  ZWV24V3T0AURI5ZCR  2017   35      F        99362   WALLA WALLA   
19248  ZWV24V3T0AURI5ZCR  2018   37      F        99362   WALLA WALLA   
19249  ZWV2AHIFLGTGTZYWM  2016   68      F        98837         GRANT   
19250  ZWV2AHIFLGTGTZYWM  2017   69      F        98837         GRANT   
19251  ZWV2F2FLESCVC1FKP  2018   64      M        99208       SPOKANE   
19252  ZWV2Q4CBHXRUQ97AL  2016   58      F        98407        PIERCE   
19253  ZWV3KNS7P0PJ6JMDY  2016   66      F        98802       DOUGLAS   
19254  ZWV43UE57CQKNIKFA  2018   71      F        98037     SNOHOMISH   
19255  ZWV6397SESED7WQ7K  2017   61      F        98902        YAKIMA   


In [9]:
# Tests if the number of claims for both reshaped and original(actual) are the same
def test(memberid, year):
    reshaped_claims = reshaped_med_data.loc[(reshaped_med_data['member_id'] == memberid) & 
                      (reshaped_med_data['year'] == year),].total_claims.item()
    print('Reshaped number of claims: ', reshaped_claims)
    
    actual = med.loc[(med['Member ID Encrypted'] == memberid) & (med['Incurred Year'] == year),]
    print('Actual number of claims: ', len(actual['Claim ID'].unique()))
    
    print(reshaped_claims == len(actual['Claim ID'].unique()))

In [10]:
test('71069K607Y1CQ9BV5', 2016)

Reshaped number of claims:  2
Actual number of claims:  2
True


In [11]:
test('71069K607Y1CQ9BV5', 2017)

Reshaped number of claims:  4
Actual number of claims:  4
True


In [12]:
test('71069K607Y1CQ9BV5', 2018)

Reshaped number of claims:  20
Actual number of claims:  20
True
